In [16]:
import pandas as pd
import sqlalchemy as sql
import numpy as np

In [2]:
server = "thesis.ca6j6heoraog.eu-central-1.rds.amazonaws.com"
engine = sql.create_engine(f"mysql+pymysql://admin:FphvsYQek4@{server}/thesis_db")


In [3]:
con = engine.connect()

In [8]:
query = """
select p.group_id,
	p.participant_id,
	sum(if(series_type_id=1,1,0)) samples_BL1,
    sum(if(series_type_id=2,1,0)) samples_Int,
    sum(if(series_type_id=3,1,0)) samples_BL2,
    sum(if(series_type_id=4,1,0)) samples_FS,
    a.group_size,
    p.cohesion
from participants p
left join interpolated_interval_data iid ON p.group_id=iid.group_id and p.participant_id = iid.participant_id
left join (Select group_id, count(1) group_size from participants group by group_id) a on a.group_id=p.group_id
where iid.interpolation_series_id=3
group by group_id, participant_id

"""

```sql
select p.group_id,
	p.participant_id,
	sum(if(series_type_id=1,1,0)) samples_BL1,
    sum(if(series_type_id=2,1,0)) samples_Int,
    sum(if(series_type_id=3,1,0)) samples_BL2,
    sum(if(series_type_id=4,1,0)) samples_FS,
    a.group_size,
    p.cohesion
from participants p
left join interpolated_interval_data iid ON p.group_id=iid.group_id and p.participant_id = iid.participant_id
left join (Select group_id, count(1) group_size from participants group by group_id) a on a.group_id=p.group_id
group by group_id, participant_id


```

In [31]:
df = pd.read_sql_query(query,con)
df

,group_id,participant_id,samples_BL1,samples_Int,samples_BL2,samples_FS,group_size,cohesion
0,1001,101,647.0,477.0,0.0,519.0,3,4.25
1,1001,102,647.0,477.0,0.0,519.0,3,5.00
2,1001,103,647.0,477.0,0.0,519.0,3,5.25
3,1003,108,589.0,583.0,592.0,475.0,2,4.00
4,1003,109,589.0,583.0,592.0,475.0,2,1.50
...,...,...,...,...,...,...,...,...
136,1050,252,622.0,633.0,617.0,555.0,3,4.50
137,1050,253,622.0,633.0,617.0,555.0,3,2.50
138,1051,254,603.0,508.0,617.0,490.0,3,3.75
139,1051,255,603.0,508.0,617.0,490.0,3,4.00


In [18]:
# remove rows where not columns have samples
df= df.loc[np.all(df.loc[:,'samples_BL1':'samples_FS'].values>10,axis=1),:]
df

,group_id,participant_id,samples_BL1,samples_Int,samples_BL2,samples_FS,group_size,cohesion
3,1003,108,589.0,583.0,592.0,475.0,2,4.00
4,1003,109,589.0,583.0,592.0,475.0,2,1.50
5,1004,110,603.0,507.0,614.0,498.0,3,3.50
6,1004,111,603.0,507.0,614.0,498.0,3,2.75
7,1004,112,603.0,507.0,614.0,498.0,3,3.75
...,...,...,...,...,...,...,...,...
136,1050,252,622.0,633.0,617.0,555.0,3,4.50
137,1050,253,622.0,633.0,617.0,555.0,3,2.50
138,1051,254,603.0,508.0,617.0,490.0,3,3.75
139,1051,255,603.0,508.0,617.0,490.0,3,4.00


In [30]:
(df.groupby('group_id')['participant_id'].count().map(lambda a: a==3 or np.nan)).dropna().index.values

array([1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014,
       1016, 1019, 1020, 1022, 1024, 1025, 1026, 1027, 1028, 1029, 1030,
       1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041,
       1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051])

In [33]:
!date

Tue 12 Jan 2021 09:27:02 IST
